* Для решения задачи по поиску перс. данных предпочла бы максимально автоматизировать процесс в одном скрипте, или можно первый анализ сделать в отдельном файле.
* Далее буду исходить, исключая биометри, из буквенно-численных ПД (персональных данных) - таких как фамилия имя отчество, телефон, адрес, СНИЛС, ИНН, паспортные данные, электронная почта, дата рождения, место рождения.
* Примерные источники в ЕХД (единое хранилище данных):
- файлы: fail1.csv, fail2.csv, fail3.csv, fail4.xlsx, fail5.xlsx, fail6.xlsx;
- таблицы базы данных postgres схема public: table1, table2, table3;
- файлы: fail7.xml, fail8.xml;
- файлы базы MongoDB: fail9.json, fail10.json, fail11.json.

* считаю, что основой должны стать регулярные выражения, 
* подключаемся ко всем источникам ЕХД,
* изучаем источники с уже описанными данными - строим функции на их основе,
* ищем принципиальное наличие данных во всех источниках, уже разработанными функциями,
* Вторая часть - это постоянное изучение вновь поступивших данных и разработка новых функций и их обогащение.

In [ ]:
#Подключение библиотек
import pandas as pd
from datetime import datetime
import os
import re
import requests
import json
import psycopg2
from pymongo import MongoClient

###   1. Подключение ко всем представленным источникам ЕХД.

In [ ]:
# Подключение к PostgreSQL
def connect_postgres():
    conn = psycopg2.connect(
        dbname='база',
        user='пользователь',
        password='пароль',
        host='localhost')
    return conn

# Подключение к MongoDB
def connect_mongodb():
    client = MongoClient('mongodb://localhost:27017/')
    return client

# Чтение файлов CSV и Excel
def read_csv_excel(file_path):
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        return pd.read_excel(file_path)

# Чтение JSON файлов MongoDB
def read_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Чтение XML файлов
def read_xml(file_path):
    # Здесь вы можете использовать библиотеку xml.etree.ElementTree или другую для обработки XML
    pass

###   2. Определение состава ЕХД.

Примерные запросы

Postgres

In [ ]:
#все схемы базы Postgres
query = ''' SELECT schema_name
           FROM information_schema.schemata;
        '''  
# просмотр всех таблиц в схеме
query = ''' SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public';
        '''
#просмотр всех столбцов конкретной таблицы
query = ''' SELECT column_name, data_type, character_maximum_length
            FROM information_schema.columns
            WHERE table_name = 'моя таблица'
        '''  

Mongo

In [ ]:
# посмотреть все базы
show dbs

# посмотреть все коллекции
use имя_базы
show collections

#узнать структуру документов
db.имя_коллекции.findOne()

###   3. Функции для проверки по регулярным выражениям

примерные функции, будем изменять после исследования данных на возможные комбинации в каждой категории: фамилия имя отчество, телефон, адрес, СНИЛС, ИНН, паспортные данные, электронная почта, дата рождения, место рождения.

In [ ]:
# Функция для проверки номера телефона
def is_valid_phone_number(phone):
    pattern = r'^(?:\+7|8|7)?\s*\(?9?15\)?[-\s]?\d{3}[-\s]?\d{2}[-\s]?\d{2}$'
    return re.match(pattern, phone)

# Другие функции для проверки персональных данных (например, email, СНИЛС и т.д.)
def is_valid_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(pattern, email)

### 4. Сохранение состояния хранилища + проверка на наличие ПД.

Сохраним состояние хранилища, чтобы впоследствие изучать только вновь поступившие данные

Примерные функции

In [ ]:
# Сохранение состояния хранилища
def save_storage_state(state, filename='storage_state.json'):
    with open(filename, 'w') as f:
        json.dump(state, f)

# Загрузка состояния хранилища
def load_storage_state(filename='storage_state.json'):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return {}

def main():
    # Подключение к базам данных
    postgres_conn = connect_postgres()
    mongo_client = connect_mongodb()

    # Получение текущего состояния хранилища
    current_state = {
        'files': os.listdir('path_to_your_files'),
        'postgres_tables': ['table1', 'table2', 'table3'],
        'mongo_collections': mongo_client['your_db'].list_collection_names()
    }

    # Загрузка предыдущего состояния хранилища
    previous_state = load_storage_state()

    # Обнаружение новых данных
    new_files = set(current_state['files']) - set(previous_state.get('files', []))
    new_postgres_tables = set(current_state['postgres_tables']) - set(previous_state.get('postgres_tables', []))
    new_mongo_collections = set(current_state['mongo_collections']) - set(previous_state.get('mongo_collections', []))

    # Анализ данных на наличие персональных данных
    for file in current_state['files']:
        if file in new_files or not previous_state:
            data = read_csv_excel(os.path.join('path_to_your_files', file))
            
    # Сохранение текущего состояния хранилища
    save_storage_state(current_state)

проверяем состав состав папки на наличие, например, файлов excel

In [ ]:
def find_new_excel_files(directory, last_checked):
    """
    Находит новые Excel-файлы в указанной директории с момента последней проверки.

    :param directory: Путь к директории для поиска файлов.
    :param last_checked: Время последней проверки.
    :return: Новые файлы и текущее время проверки.
    """
    # Список для хранения новых файлов
    new_files = []

    # Получаем текущее время для обновления времени последней проверки
    current_time = datetime.now()

    # Проходим по всем файлам в директории
    for filename in os.listdir(directory):
        # Проверяем, является ли файл Excel-файлом (по расширению)
        if filename.endswith('.xlsx') or filename.endswith('.xls'):
            # Получаем полный путь к файлу
            file_path = os.path.join(directory, filename)
            # Получаем время последнего изменения файла
            file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))
            # Если файл был изменен после последней проверки, добавляем его в список
            if file_mod_time > last_checked:
                new_files.append(filename)

    return new_files, current_time

# Пример
directory_path = '/путь/к/папке'
last_checked_time = datetime(2023, 10, 1)  # Установим начальную дату последней проверки

new_files, last_checked_time = find_new_excel_files(directory_path, last_checked_time)
if new_files:
    print("Новые файлы:", new_files)
else:
    print("Новых файлов не обнаружено.")